<a href="https://colab.research.google.com/github/tyro2001/hello-world/blob/master/ML1_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import scipy.stats as ss

**Problem 1**

In [2]:
NUM_DRAWS = 1_000_000

draws = ss.expon.rvs(size=NUM_DRAWS)
results = {'MSE': dict(), "MAE" : dict()}
estimators = {'Mean  ': 1.0, 'Median': math.log(2)}

for name, y_hat in estimators.items():
    residuals = draws - y_hat
    rss = np.sum(np.power(residuals, 2))
    rsa = np.sum(np.abs(residuals))
    results['MSE'][name] = rss / NUM_DRAWS
    results['MAE'][name] = rsa / NUM_DRAWS

for measure in results:
    for name, result in results[measure].items():
        print(measure, name, result)
    print()




MSE Mean   1.0018393582898109
MSE Median 1.0958504205323436

MAE Mean   0.7356608484996945
MAE Median 0.6927683839800061



The mean works better for mean squared error (MSE), but the median is better for mean absolute error (MAE). 

**Problem 2**

In [ ]:
def loss(a, b, z):
    assert(a > 0)
    assert(b > 0)
    scaled_err = a * (y - y_hat)
    return b * (math.exp(a*z) - a*z - 1)

